# Baselines

In [1]:
# helper functions
from FLutils import (
    load_df,               # load data
    #create_keras_model,    # construct a deep neural network (keras)
    #model_fn,              # convert keras model to tff.learning.models
    #prep_fed_train,        # convert training data to tensors for learning with tensorflow
    #prep_fed_test,         # convert test data to tensors for testing with tensorflow (other format than training data)
    #train_model,           # train a keras model
    #train_fed              # train a keras model federated with distributed data
    )


2023-06-20 16:52:18.849788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 16:52:19.622825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
# ingest data
# ===========

df_paths = [
    '../output/data/insurance-clean.csv',
    "https://raw.githubusercontent.com/Olhaau/fl-official-statistics-addon/main/output/data/insurance-clean.csv"
]

df = load_df(df_paths)
df.head(3)

loaded data from ../output/data/insurance-clean.csv


,age,sex,bmi,children,smoker,region,charges,region0,region1,region2,region3
0,0.021739,0.0,0.321227,0.0,1.0,southwest,16884.9240,0.0,0.0,0.0,1.0
1,0.000000,1.0,0.479150,0.2,0.0,southeast,1725.5523,0.0,0.0,1.0,0.0
2,0.217391,1.0,0.458434,0.6,0.0,southeast,4449.4620,0.0,0.0,1.0,0.0


In [22]:
# select features, target (first column) and clients
# ==================================================

# features for centralized learning
features = ['age', 'sex', 'bmi', 'children', 'smoker'
            , 'region0', 'region1', 'region2', 'region3']

target = 'charges'

# features for federated learning
features_fed = features[:5]

# clients for data splits
clients = df.region.unique()

# show target + features in the data
df_ml = df.loc[:, [target] + features ]
df_ml.head(3)

,charges,age,sex,bmi,children,smoker,region0,region1,region2,region3
0,16884.9240,0.021739,0.0,0.321227,0.0,1.0,0.0,0.0,0.0,1.0
1,1725.5523,0.000000,1.0,0.479150,0.2,0.0,0.0,0.0,1.0,0.0
2,4449.4620,0.217391,1.0,0.458434,0.6,0.0,0.0,0.0,1.0,0.0


In [25]:
from pycaret import *
from pycaret.regression import *


In [23]:
df_ml

,charges,age,sex,bmi,children,smoker,region0,region1,region2,region3
0,16884.92400,0.021739,0.0,0.321227,0.0,1.0,0.0,0.0,0.0,1.0
1,1725.55230,0.000000,1.0,0.479150,0.2,0.0,0.0,0.0,1.0,0.0
2,4449.46200,0.217391,1.0,0.458434,0.6,0.0,0.0,0.0,1.0,0.0
3,21984.47061,0.326087,1.0,0.181464,0.0,0.0,0.0,1.0,0.0,0.0
4,3866.85520,0.304348,1.0,0.347592,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1333,10600.54830,0.695652,1.0,0.403820,0.6,0.0,0.0,1.0,0.0,0.0
1334,2205.98080,0.000000,0.0,0.429379,0.0,0.0,1.0,0.0,0.0,0.0
1335,1629.83350,0.000000,0.0,0.562012,0.0,0.0,0.0,0.0,1.0,0.0
1336,2007.94500,0.065217,0.0,0.264730,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
nreps, nfolds = 5, 5
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold

evaluation = RepeatedStratifiedKFold(n_splits = nfolds, n_repeats = nreps#, random_state = 42
                                     )


In [41]:
clf = setup(df_ml, target = target, session_id = 1234, preprocess=False
            #,fold_strategy = evaluation
            )

,Description,Value
0,Session id,1234
1,Target,charges
2,Target type,Regression
3,Original data shape,"(1338, 10)"
4,Transformed data shape,"(1338, 10)"
5,Transformed train set shape,"(936, 10)"
6,Transformed test set shape,"(402, 10)"
7,Numeric features,9


In [42]:
# benchmark experiment
best = compare_models(
    sort = "R2", fold = 5#, budget_time = 5
)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2486.1258,20889992.8229,4517.1081,0.8469,0.4274,0.2927,0.0190
lightgbm,Light Gradient Boosting Machine,2850.2220,23583316.4520,4813.5289,0.8281,0.5439,0.3620,0.0120
catboost,CatBoost Regressor,2718.2365,23572477.4172,4816.4303,0.8276,0.4743,0.3334,0.0110
rf,Random Forest Regressor,2659.6378,23453477.1832,4800.4334,0.8270,0.4656,0.3346,0.0210
xgboost,Extreme Gradient Boosting,3001.2314,27562150.4000,5216.2469,0.7996,0.5723,0.3903,0.0170
et,Extra Trees Regressor,2732.7145,27421662.1402,5183.8911,0.7974,0.4796,0.3249,0.0210
ada,AdaBoost Regressor,4147.9272,27709232.4756,5213.2588,0.7957,0.6219,0.7134,0.0120
knn,K Neighbors Regressor,3724.2971,36469640.4000,5988.9230,0.7405,0.4922,0.3633,0.0110
lr,Linear Regression,4135.6304,37392039.0000,6064.7804,0.7355,0.5683,0.4167,0.0190
ridge,Ridge Regression,4131.9550,37462317.4000,6068.7744,0.7354,0.5606,0.4145,0.0120


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [43]:
lr = create_model('lr')

tuned_lr = tune_model(lr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4983.3354,58307780.0000,7635.9531,0.7570,0.4505,0.3197
1,4020.2007,33874264.0000,5820.1602,0.7263,0.5171,0.3696
2,4059.1189,37969388.0000,6161.9307,0.7514,0.4967,0.3925
3,4103.8110,32285642.0000,5682.0454,0.7761,0.6382,0.4356
4,5021.6201,48694812.0000,6978.1670,0.6767,0.5986,0.4801
5,4171.3735,33794948.0000,5813.3423,0.7953,0.6225,0.4547
6,4247.9131,44902724.0000,6700.9497,0.5415,0.5977,0.4388
7,3753.0078,31527774.0000,5614.9600,0.7972,0.7180,0.3489
8,3214.6860,23658410.0000,4863.9912,0.8074,0.5167,0.4555


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4983.3354,58307780.0000,7635.9531,0.7570,0.4505,0.3197
1,4020.2007,33874264.0000,5820.1602,0.7263,0.5171,0.3696
2,4059.1189,37969388.0000,6161.9307,0.7514,0.4967,0.3925
3,4103.8110,32285642.0000,5682.0454,0.7761,0.6382,0.4356
4,5021.6201,48694812.0000,6978.1670,0.6767,0.5986,0.4801
5,4171.3735,33794948.0000,5813.3423,0.7953,0.6225,0.4547
6,4247.9131,44902724.0000,6700.9497,0.5415,0.5977,0.4388
7,3753.0078,31527774.0000,5614.9600,0.7972,0.7180,0.3489
8,3214.6860,23658410.0000,4863.9912,0.8074,0.5167,0.4555


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [44]:
print(tuned_lr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, positive=False)


In [45]:
rf = create_model('rf')

tuned_rf = tune_model(rf)
print(rf)
print(tuned_rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3006.5775,29517795.0038,5433.0282,0.8770,0.4115,0.2791
1,2721.8961,26139711.4309,5112.7010,0.7888,0.4585,0.3216
2,2670.9540,26032652.6580,5102.2204,0.8296,0.4025,0.3251
3,2430.5120,15761897.1264,3970.1256,0.8907,0.4436,0.3247
4,2697.0849,24546953.2220,4954.4882,0.8370,0.4427,0.2929
5,2352.5778,16824183.6630,4101.7294,0.8981,0.4382,0.3272
6,3332.9366,36550214.5249,6045.6773,0.6268,0.5835,0.3715
7,2172.0071,16644586.3688,4079.7777,0.8930,0.4255,0.2364
8,2861.5890,23817333.6518,4880.3006,0.8061,0.5768,0.5436


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3390.7022,34844625.5454,5902.9336,0.8548,0.4089,0.3034
1,2866.3126,21680926.1754,4656.2781,0.8248,0.4840,0.4045
2,2917.3160,25806098.1256,5079.9703,0.8310,0.4144,0.3705
3,2533.2484,12408119.6210,3522.5161,0.9139,0.4557,0.4152
4,3245.0803,26845629.5086,5181.2768,0.8218,0.5180,0.4502
5,2894.6246,17928480.6194,4234.2037,0.8914,0.4812,0.4501
6,3312.7199,33789280.1575,5812.8547,0.6550,0.5340,0.3980
7,2361.2886,14722207.9950,3836.9530,0.9053,0.4210,0.3092
8,3043.8175,20105287.5486,4483.8920,0.8363,0.6000,0.6328


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='squared_error',
                      max_depth=None, max_features=1.0, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=1234, verbose=0,
                      warm_start=False)
RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='squared_error',
                      max_depth=7, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.5,
                      min_samples_leaf=2, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                      oob_score=False, random_state=1234, verbose=0,
                      

In [37]:
plot_model(best, plot = 'error')


ValueError: Estimator [] does not have the required fit() method.

In [31]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [9]:
# load sample dataset
from pycaret.datasets import get_data
data = get_data('diabetes')

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
from pycaret.classification import *
s = setup(data, target = 'Class variable', session_id = 123)

,Description,Value
0,Session id,123
1,Target,Class variable
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [11]:
# from pycaret.classification import ClassificationExperiment
# s = ClassificationExperiment()
# s.setup(data, target = 'Class variable', session_id = 123)

,Description,Value
0,Session id,123
1,Target,Class variable
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [12]:
# functional API
best = compare_models()

# OOP API
#best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7689,0.8047,0.5602,0.7208,0.6279,0.4641,0.4736,0.2520
ridge,Ridge Classifier,0.7670,0.0000,0.5497,0.7235,0.6221,0.4581,0.4690,0.0090
lda,Linear Discriminant Analysis,0.7670,0.8055,0.5550,0.7202,0.6243,0.4594,0.4695,0.0110
rf,Random Forest Classifier,0.7485,0.7911,0.5284,0.6811,0.5924,0.4150,0.4238,0.0800
nb,Naive Bayes,0.7427,0.7955,0.5702,0.6543,0.6043,0.4156,0.4215,0.0100
catboost,CatBoost Classifier,0.7410,0.7993,0.5278,0.6630,0.5851,0.4005,0.4078,1.1770
gbc,Gradient Boosting Classifier,0.7373,0.7917,0.5550,0.6445,0.5931,0.4013,0.4059,0.0360
ada,Ada Boost Classifier,0.7372,0.7799,0.5275,0.6585,0.5796,0.3926,0.4017,0.0260
et,Extra Trees Classifier,0.7299,0.7788,0.4965,0.6516,0.5596,0.3706,0.3802,0.0720
qda,Quadratic Discriminant Analysis,0.7282,0.7894,0.5281,0.6558,0.5736,0.3785,0.3910,0.0100


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7689,0.8047,0.5602,0.7208,0.6279,0.4641,0.4736,0.0150
ridge,Ridge Classifier,0.7670,0.0000,0.5497,0.7235,0.6221,0.4581,0.4690,0.0090
lda,Linear Discriminant Analysis,0.7670,0.8055,0.5550,0.7202,0.6243,0.4594,0.4695,0.0110
rf,Random Forest Classifier,0.7485,0.7911,0.5284,0.6811,0.5924,0.4150,0.4238,0.0330
nb,Naive Bayes,0.7427,0.7955,0.5702,0.6543,0.6043,0.4156,0.4215,0.0110
catboost,CatBoost Classifier,0.7410,0.7993,0.5278,0.6630,0.5851,0.4005,0.4078,0.0410
gbc,Gradient Boosting Classifier,0.7373,0.7917,0.5550,0.6445,0.5931,0.4013,0.4059,0.0240
ada,Ada Boost Classifier,0.7372,0.7799,0.5275,0.6585,0.5796,0.3926,0.4017,0.0320
et,Extra Trees Classifier,0.7299,0.7788,0.4965,0.6516,0.5596,0.3706,0.3802,0.0390
qda,Quadratic Discriminant Analysis,0.7282,0.7894,0.5281,0.6558,0.5736,0.3785,0.3910,0.0100


Processing:   0%|          | 0/69 [00:00<?, ?it/s]